In [99]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd  # for data analytics

# The two packages below are for plotting
import seaborn as sns  #seaborn has nice features. Need to load pyplot with seaborn at the same time
import matplotlib.pyplot as plt
%matplotlib inline

import re

In [3]:
df = pd.read_csv('US_Accidents_Dec21_updated[1].csv')

In [19]:
df = df.dropna(subset = ['Street'])

In [100]:
street_ls = []   # Correct inconsistent direction spelling for "Street", capitalize
for mystr in df["Street"]:
    mystr = str(mystr).upper().strip()
    newstr = mystr.replace('NORTH', 'N')
    newstr = newstr.replace('SOUTH', 'S')
    newstr = newstr.replace('WEST', 'W')
    street_ls.append(newstr)
print(street_ls[0:50])

['OUTERBELT E', 'I-70 E', 'I-75 S', 'I-77 N', 'I-75 S', 'STATE ROUTE 32', 'I-75 S', 'I-71 S', 'E HANTHORN RD', 'OUTERBELT W', 'I-75 N', 'HOPPLE ST', 'I-90 E', 'I-71 N', 'TITUS RD', 'CADIZ RD', 'OUTERBELT N', 'OUTERBELT S', 'N FWY S', 'I-75 S', 'US-33 S', 'I-75 S', 'N FWY N', 'I-70 E', 'I-75 S', 'I-65 S', 'I-675 S', 'I-70 E', 'US-421 S', 'I-74 W', 'US-421 S', 'US-35 E', 'US-50 E', 'LONG LAKE LN', 'US-23 N', 'I-65 S', 'US-23 N', 'I-279 N', 'US-19 TRK S', 'COLUMBUS PIKE', 'NW FWY W', 'I-76 E', 'W STATE ROAD 26', 'N MERIDIAN ST', 'S STATE ROAD 1', 'S STATE ROAD 1', 'W INNERBELT N', 'STATE ROUTE 49', 'I-70 W', 'I-90 E']


In [28]:
df = df.drop('Street', axis=1)
df["Street"] = street_ls

In [106]:
df.Street.value_counts().nlargest(10)  #

I-95 N     39853
I-5 N      39402
I-95 S     36425
I-5 S      30229
I-10 E     26164
I-10 W     25298
I-80 W     17545
I-80 E     16873
I-405 N    13708
I-15 N     12675
Name: Street, dtype: int64

In [32]:
desc_ls = []   # Correct inconsistent direction spelling for "Cross_street", capitalize
for mystr in df["Description"]:
    mystr = str(mystr).upper().strip()
    newstr = mystr.replace('NORTH', 'N')
    newstr = newstr.replace('SOUTH', 'S')
    newstr = newstr.replace('WEST', 'W')
    desc_ls.append(newstr)
print(desc_ls[0:10])

['BETWEEN SAWMILL RD/EXIT 20 AND OH-315/OLENTANGY RIV RD/EXIT 22 - ACCIDENT.', 'AT OH-4/OH-235/EXIT 41 - ACCIDENT.', 'AT I-71/US-50/EXIT 1 - ACCIDENT.', 'AT DART AVE/EXIT 21 - ACCIDENT.', 'AT MITCHELL AVE/EXIT 6 - ACCIDENT.', 'AT DELA PALMA RD - ACCIDENT.', 'AT OH-4/EXIT 54 - ACCIDENT.', 'AT BAGLEY RD/EXIT 235 - ACCIDENT.', 'AT OH-65/EXIT 122 - ACCIDENT.', 'AT I-71/EXIT 26 - ACCIDENT.']


In [33]:
df = df.drop('Description', axis=1)
df["Description"] = desc_ls

In [104]:
df.Description.value_counts().nlargest(20)  # Not every description has cross street information. 

A CRASH HAS OCCURRED CAUSING NO TO MINIMUM DELAYS. USE CAUTION.                                  7978
A CRASH HAS OCCURRED USE CAUTION.                                                                2531
AN UNCONFIRMED REPORT OF A CRASH HAS BEEN RECEIVED. USE CAUTION.                                 2308
HAZARDOUS DEBRIS IS CAUSING NO TO MINIMUM DELAYS. USE CAUTION.                                   2095
AT I-15 - ACCIDENT.                                                                              2070
A DISABLED VEHICLE IS CREATING A HAZARD CAUSING NO TO MINIMUM DELAYS. USE CAUTION.               1912
AT I-5 - ACCIDENT.                                                                               1907
AT I-405/SAN DIEGO FWY - ACCIDENT.                                                               1769
AT I-605 - ACCIDENT.                                                                             1486
INCIDENT ON I-95 NB NEAR I-95 DRIVE WITH CAUTION.                                 

In [101]:
desc_test_df = df[df['Description'] == "AT I-405/SAN DIEGO FWY - ACCIDENT."]
desc_test_df.Street.value_counts().nlargest(10)

I-405 N           194
I-405 S           148
US-101 S          141
I-10 E            133
WILSHIRE BLVD      94
CA-55 N            79
US-101 N           79
CA-55 S            61
W DEL AMO BLVD     58
SEPULVEDA BLVD     52
Name: Street, dtype: int64

In [105]:
desc_test_df = df[df['Description'] == "AT I-15 - ACCIDENT."]
desc_test_df.Street.value_counts().nlargest(10)

CA-91 W            519
CA-91 E            356
I-10 E             103
I-10 W              83
CA-60 W             67
CA-60 E             62
CA-78 E             58
RIVERSIDE FWY E     40
S STATE ST          39
W 3300 S            37
Name: Street, dtype: int64

#### Parsing street names (quick and dirty)

In [112]:
print(street_ls[0:20])
len(street_ls)

['OUTERBELT E', 'I-70 E', 'I-75 S', 'I-77 N', 'I-75 S', 'STATE ROUTE 32', 'I-75 S', 'I-71 S', 'E HANTHORN RD', 'OUTERBELT W', 'I-75 N', 'HOPPLE ST', 'I-90 E', 'I-71 N', 'TITUS RD', 'CADIZ RD', 'OUTERBELT N', 'OUTERBELT S', 'N FWY S', 'I-75 S']


2845340

In [118]:
des_list = []

i = 0
for mystr in df["Description"]:
    mystr = ' '+mystr+' '   # add space before and after the string, so short test delimiter can work
    #print(mystr)
    
    streetname = street_ls[i].upper()
    delimiter = street_ls[i] + '| ON |\*|\.| IN | IS | CLOSED | A | \(| \[| TO | BETWEEN | AND | AT | BUS | SFPD | @ | TOWN | DMV | SUV | # | APPROACHING | UNKNOWN | POI | HEAD |\
                      - |CLOSED AT |ROAD CLOSED AT |CLOSED BETWEEN |/| BOTH | DIRECTIONS |\
                      INCIDENT ON |SLOW TRAFFIC ON |STATIONARY TRAFFIC ON |INCIDENT|SLOW TRAFFIC|CONSTRUCTION|NOTIFICATION\]|MULTI VEHICLE|\
                      SLOW TRAFFIC FROM |STATIONARY TRAFFIC FROM | FROM |HAZARDOUS DEBRIS|MOTOR VEHICLE|VEHICLE|MULTIPLE|\
                      SLOW TRAFFIC AT | NEAR |RAMP CLOSED |RAMP | AT | PAST |\
                      ACCIDENTS |STATIONARY TRAFFIC AT |CLOSED ROAD |QUEUING TRAFFIC | FROM |LANDSLIDE HAS OCCURRED|\
                      RAMP TO | ACCIDENT |NYSDOT|NJ DOT|CONNDOT:|CRASH|EXIT|HAS OCCURRED USE CAUTION.|\
                      MOTOR VEHICLE ACCIDENT|CRASH ON|DUE|LEFT LANE BLOCKED|AN ABANDONED VEHICLE|\
                      ALL LANES BLOCKED|NO OTHER VEHS INV|DISABLED VEHICLE|DISABLED STALL|STALL HAS THE LEFT LANE|STALL HAS THE RIGHT LANE|\
                      STALL HAS BEEN REPORTED|VEHICLE HAS STALLED|COLLISION REPORTED|COLLISION|\
                      ROCK FALL HAS OCCURRED|POLICE ACTIVITY HAS WB|\
                      VEHICLE HAS STALLED SB|\
                      HAS BOTH LANES BLOCKED|\
                      VEHICLE HAS STALLED NB|:'
    #print(delimiter)
    
    newstr = re.split(delimiter, mystr.upper())   
    #print(newstr)

    # When the splited string list is longer than 3
    if len(newstr) >= 3:
        
        ## when the 1st string is empty
        if newstr[0].strip() == '':  ### when the 1st string is empty
            #print(newstr[1])
            
            ### when the 2nd string is empty
            if newstr[1].strip() == '':  ### when the 2nd string is empty
                
                #### if the 3rd string between (0:5] and last letter is not a number OR it is all number, not a street name
                if len(newstr[2].strip()) <= 5 and len(newstr[2].strip()) > 0 and (newstr[2].strip()[-1].isdigit() == False or re.match("^[0-9 ]+$",newstr[2])): 
                    des_list.append(np.nan)
                    
                #### when the 3rd string is longer than 23, not a street name    
                elif len(newstr[2].strip()) > 23:
                    des_list.append(np.nan)
                    
                #### if the 3rd string is between [3, 23], likely a street name, if it has only 3 letters, the last letter is a number, e.g., I-5    
                elif len(newstr[2].strip()) <= 23 and len(newstr[2].strip())>= 3:
                    newstr_new = ' '+newstr[2].strip()+' '  # Add space for delimiter, remove unwanted texts
                    newstr_1 = re.split(' THE | BEFORE | BEYOND | NEAR | FROM | ON | IN | IS | A | \(| \[| TO | BETWEEN | AND | AT | - ACCIDENT| - ROAD | - |', newstr_new)
                    
                    if newstr_1[0].strip() != '':
                        if len(newstr_1[0].strip())>= 5:
                            if re.match("^[0-9 ]+$",newstr_1[0]):
                                des_list.append(np.nan)
                            else:
                                des_list.append(newstr_1[0].upper().strip())
                        elif len(newstr_1[0].strip()) <= 5 and newstr_1[1].strip()[-1].isdigit():
                            if re.match("^[0-9 ]+$",newstr_1[0]):
                                des_list.append(np.nan)
                            else:
                                des_list.append(newstr_1[0].upper().strip())
                        else:
                            des_list.append(np.nan)
                    
                    elif len(newstr_1) > 1 and newstr_1[1].strip() != '':
                        if len(newstr_1[1].strip()) <= 5 and newstr_1[1].strip()[-1].isdigit():
                            if re.match("^[0-9 ]+$",newstr_1[1]):
                                des_list.append(np.nan)
                            else:
                                des_list.append(newstr_1[1].upper().strip())
                                                
                        elif len(newstr_1[1].strip())> 5:
                            if re.match("^[0-9 ]+$",newstr_1[1]):
                                des_list.append(np.nan)
                            else:
                                des_list.append(newstr_1[0].upper().strip())
                        else:
                            des_list.append(np.nan)
                    
                    else:
                        des_list.append(np.nan)    # otherwise add NaN
                
                #### when the 3rd string is not between (0,5], [2,23] or >23, add NaN
                else:                       
                    des_list.append(np.nan)
                    
            
            ### when the 2nd string is not empty
            else:
                
                #### when the 2nd string is between (0, 3] and last letter is not a number, not a street, add as NaN
                if len(newstr[1].strip())<= 5 and len(newstr[1].strip()) > 0 and (newstr[1].strip()[-1].isdigit() == False or re.match("^[0-9 ]+$",newstr[1])):
                    #print(mystr)
                    #print(newstr)
                    des_list.append(np.nan)
                
                #### when the 2nd string is longer than 23, not a street name, add as NaN
                elif len(newstr[1].strip()) > 23:
                    des_list.append(np.nan)
                
                #### when the 2nd string is between [2,23], likely a street name
                elif len(newstr[1].strip()) <= 23 and len(newstr[1].strip())>= 3:
                    newstr_new = ' '+newstr[1].strip()+' '
                    newstr_1 = re.split(' THE | BEFORE | BEYOND | NEAR | FROM | ON | IN | IS | A | \(| \[| TO | BETWEEN | AND | AT | - ACCIDENT| - ROAD | - ', newstr_new)
                    #print(newstr_1)
                    
                    if newstr_1[0].strip() != '':
                        if len(newstr_1[0].strip())> 5: #and newstr_1[0].strip()[-1].isdigit():
                            if re.match("^[0-9 ]+$",newstr_1[0]):
                                des_list.append(np.nan)
                            else:
                                des_list.append(newstr_1[0].upper().strip())
                        elif len(newstr_1[0].strip()) <= 5 and newstr_1[0].strip()[-1].isdigit():
                            if re.match("^[0-9 ]+$",newstr_1[0]):
                                des_list.append(np.nan)
                            else:
                                des_list.append(newstr_1[0].upper().strip())
                        else:
                            des_list.append(np.nan)
                    
                    elif len(newstr_1)>1  and newstr_1[1].strip() != '':
                        if len(newstr_1[1].strip()) <= 5 and newstr_1[1].strip()[-1].isdigit():
                            if re.match("^[0-9 ]+$",newstr_1[1]):
                                des_list.append(np.nan)
                            else:
                                des_list.append(newstr_1[1].upper().strip())
                                                
                        elif len(newstr_1[1].strip())> 5:
                            if re.match("^[0-9 ]+$",newstr_1[1]):
                                des_list.append(np.nan)
                            else:
                                des_list.append(newstr_1[1].upper().strip())
                        else:
                            des_list.append(np.nan)
                    else:
                        des_list.append(np.nan)
                
                #### when the 2nd string is not between (0,5], [2,23] or >23, add NaN
                else:
                    des_list.append(np.nan)
        
        ## when the 1st string is not empty
        else:
            
            #### when the 1st string is between (0, 3] and last letter is not a number, not a street, add as NaN
            if len(newstr[0].strip()) <= 5 and len(newstr[0].strip()) > 0 and (newstr[0][-1].isdigit() == False or re.match("^[0-9 ]+$",newstr[0])):
                #print(mystr)
                #print(newstr)
                des_list.append(np.nan)
            
            #### when the 1st string is longer than 23, not a street name, add as NaN
            elif len(newstr[0].strip()) > 23:
                des_list.append(np.nan)
            
            #### when the 1st string is between [2,23], likely a street name
            elif len(newstr[0].strip()) <= 23 and len(newstr[0].strip())>= 3:
                newstr_new = ' '+newstr[0].strip()+' '
                newstr_1 = re.split(' THE | BEFORE | BEYOND | NEAR | FROM | ON | IN | IS | A | \(| \[| TO | BETWEEN | AND | AT | - ACCIDENT| - ROAD | - ', newstr_new)
                #print(newstr_1)
                
                if newstr_1[0].strip() != '':
                    if len(newstr_1[0].strip()) > 5: #and newstr_1[0].strip()[-1].isdigit():
                        if re.match("^[0-9 ]+$",newstr_1[0]):
                            des_list.append(np.nan)
                        else:
                            des_list.append(newstr_1[0].upper().strip())
                    elif len(newstr_1[0].strip()) <= 5 and newstr_1[0].strip()[-1].isdigit():
                        if re.match("^[0-9 ]+$",newstr_1[0]):
                            des_list.append(np.nan)
                        else:
                            des_list.append(newstr_1[0].upper().strip())
                    else:
                        des_list.append(np.nan)
                    
                elif len(newstr_1)>1 and newstr_1[1].strip() != '':
                    if len(newstr_1[1].strip()) <= 5 and newstr_1[1].strip()[-1].isdigit():
                        if re.match("^[0-9 ]+$",newstr_1[1]):
                            des_list.append(np.nan)
                        else:
                            des_list.append(newstr_1[1].upper().strip())
                                                
                    elif len(newstr_1[1].strip()) > 5:
                        if re.match("^[0-9 ]+$",newstr_1[1]):
                            des_list.append(np.nan)
                        else:
                            des_list.append(newstr_1[1].upper().strip())
                            
                    else:
                        des_list.append(np.nan)
                
                else:
                    des_list.append(np.nan)
            
            #### when the 1st string is not between (0,5], [2,23] or >23, add NaN
            else:
                des_list.append(np.nan)
                
    # when the splited string list is shorter than 3       
    else:
        if newstr[0].strip() != '':
            if len(newstr[0].strip())> 5 and len(newstr[0].strip()) <= 23:
                des_list.append(newstr[0].upper().strip())
            elif len(newstr[0].strip()) <= 5 and newstr[0].strip()[-1].isdigit():
                if re.match("^[0-9 ]+$",newstr[0]):
                    des_list.append(np.nan)
                else:
                    des_list.append(newstr[0].upper().strip())
            else:
                des_list.append(np.nan)
        else:
            des_list.append(np.nan)
    
    
    if i%100000 == 0:  #calc status output
        print(str(i) + " instances has been processed")
    i = i+1

0 instances has been processed
100000 instances has been processed
200000 instances has been processed
300000 instances has been processed
400000 instances has been processed
500000 instances has been processed
600000 instances has been processed
700000 instances has been processed
800000 instances has been processed
900000 instances has been processed
1000000 instances has been processed
1100000 instances has been processed
1200000 instances has been processed
1300000 instances has been processed
1400000 instances has been processed
1500000 instances has been processed
1600000 instances has been processed
1700000 instances has been processed
1800000 instances has been processed
1900000 instances has been processed
2000000 instances has been processed
2100000 instances has been processed
2200000 instances has been processed
2300000 instances has been processed
2400000 instances has been processed
2500000 instances has been processed
2600000 instances has been processed
2700000 instance

In [119]:
df["Cross_street"] = des_list  # add Cross_street to df

In [120]:
print(df.shape)

(2845340, 48)


In [121]:
df_new = df.dropna(subset = ['Cross_street','Street']) # remove rows with NaN value in Cross_street and Street column

In [122]:
df_new.shape

(1390073, 48)

In [123]:
df_new.to_csv('national_clean_des.csv') # save df_new as csv

#### Check parcing performance for the long street names

In [124]:
df_check = pd.DataFrame()
df_check['length'] = df_new['Cross_street'].str.len()
df_check['content']=df_new['Cross_street']
df_check.sort_values('length', ascending=False, inplace=True)
df_check = df_check.astype({"length": int})

In [125]:
df_check[df_check['length'] > 20].value_counts().nsmallest(30) #long street names not completely clean

length  content                
22      VEH TC'D INTO RP'S DOG     1
        VEH ROLLED OVER STRUCK     1
        NB US101 JSO TWIN OAKS     1
23      1 VEH POSS NOT DRIVALBE    1
22      PER MTY COM - FIRE UTL     1
        PER SRCC POSS NECK INJ     1
        VEH VS RP'S PARKED VEH     1
        OCCD NORWALK X SLAUSON     1
        VEH VS KCSO PATROL VEH     1
        NOGALES JWO PATHFINDER     1
        NORMANDIE JSO OAKHEATH     1
        OCCD NB I5 JSO ROXFORD     1
        VEH VS SEMI VEH HAULER     1
        PER EDISON 30-89244 OR     1
        VEH VS TWO PARKED VEHS     1
        OCCD SB 405 JNO CARSON     1
        XST POINT SAN PEDRO RD     1
        VEH WAS 23103 AFTER TC     1
        VETERANS MEMORIAL PKWY     1
        VICT GRY NISS ALTIMA R     1
        OCCD SB 405 VIC SUNSET     1
        NB US101 JNO MATTIE RD     1
        WB 4 JWO CUMMINGS SKWY     1
        VEH MAKING LOUD NOISES     1
        PLOT OF YUM YUM DONUTS     1
        NB SR99 JSO W 16TH ONR     1
      

In [126]:
df_check[df_check['length'] <= 4].value_counts().nsmallest(30) #short street names not completely clean

length  content
3       A21        1
4       EB91       1
3       M99        1
4       -395       1
        FL-4       1
3       D12        1
4       CR 8       1
        FL-5       1
3       #59        1
        I40        1
        I15        1
4       AR-4       1
        X 99       1
3       I26        1
4       CH-4       1
        E 20       1
        US17       1
        CR25       1
        US15       1
3       SB5        1
4       US13       1
3       R97        1
        CR1        1
        MM1        1
4       CR27       1
        US52       1
        WB 9       1
        CR85       1
        CP-7       1
        CRJ2       1
dtype: int64

#### Further cleaning by removing small counts

In [149]:
sub_df_new = df_new[df_new.groupby('Cross_street').Cross_street.transform('count')>=3].copy()

In [150]:
df_check_sub = pd.DataFrame()   #check performance after deleting small counts items
df_check_sub['length'] = sub_df_new['Cross_street'].str.len()
df_check_sub['content']=sub_df_new['Cross_street']
df_check_sub.sort_values('length', ascending=False, inplace=True)
df_check_sub = df_check_sub.astype({"length": int})

In [151]:
df_check_sub[df_check_sub['length'] <= 4].value_counts().nsmallest(30) #No junk after deleting observation < 3

length  content
4       AB 5       3
        VA-4       3
        AL-8       3
        US44       3
        ME-4       3
        DE-9       3
        EB 1       3
        MM47       3
        M-83       3
        MO-6       3
        MS-8       3
        NC-3       3
        M-65       3
        NF-3       3
        NY-2       3
        NY-3       3
3       I 9        3
4       IN-8       3
        RT 4       3
        RT 5       3
        IL-2       3
        ID-7       3
        ME-5       3
        RT34       3
        ID-1       3
        SR29       3
        SR74       3
        SR90       3
        SR92       3
        GA-4       3
dtype: int64

In [152]:
df_check_sub[df_check_sub['length'] > 20].value_counts().nsmallest(50) #No junk after deleting observation < 3

length  content                
22      SR60 W EO PIPELINE AVE     3
23      SR91 E WO ARLINGTON AVE    3
22      SR60 W EO VINEYARD AVE     3
        SR91 E WO MAGNOLIA AVE     3
23      SR78 W WOODLAND PKWY EO    3
        WB SR138 JEO 233RD ST E    3
        ST PETERSBURG REST AREA    3
22      STATE ROUTE 109 NBOUND     3
23      SR91 E WO PROMENADE AVE    3
        W FLORIDA REPUBLIC PKWY    3
        WB 80 JEO SAN PABLO DAM    3
        SR91 W WO ARLINGTON AVE    3
        WB 134 JEO ORANGE GROVE    3
        BEAR MOUNTAIN STATE PKY    3
        SR60 W WO RUBIDOUX BLVD    3
        HATFIELD-SOUDERTON PIKE    3
        I15 N TEMECULA NO(SR79)    3
        I215 N SO PLACENTIA AVE    3
        I215 N SO WASHINGTON ST    3
        I5 S DEL MAR HEIGHTS NO    3
        I5 S DEL MAR HEIGHTS SO    3
        I10 W MOUNTAIN VIEW AVE    3
        I10 E WO TIPPECANOE AVE    3
        MCNEILL-STEEP HOLLOW RD    3
        HUTCHINSON RIVER PKWY S    3
        NB 14 JSO GOLDEN VALLEY    3
      

In [153]:
sub_df_new.to_csv('national_clean_des_xstreet_above_3.csv') #output to csv

#### Stats for national data before cleaning

In [132]:
df.Severity.value_counts()
severity_2, severity_3, severity_4, severity_1 = df.Severity.value_counts()
total = len(df.Severity)
print (total)
print("Severity 1: ", severity_1, "{0:.2%} of total instances".format(severity_1/total))
print("Severity 2: ", severity_2, "{0:.2%} of total instances".format(severity_2/total))
print("Severity 3: ", severity_3, "{0:.2%} of total instances".format(severity_3/total))
print("Severity 4: ", severity_4, "{0:.2%} of total instances".format(severity_4/total))

2845340
Severity 1:  26053 0.92% of total instances
Severity 2:  2532989 89.02% of total instances
Severity 3:  155105 5.45% of total instances
Severity 4:  131193 4.61% of total instances


#### Stats for national data after cleaning

In [133]:
df_new.Severity.value_counts()
severity_2, severity_3, severity_4, severity_1 = df_new.Severity.value_counts()
total = len(df_new.Severity)
print (total)
print("Severity 1: ", severity_1, "{0:.2%} of total instances".format(severity_1/total))
print("Severity 2: ", severity_2, "{0:.2%} of total instances".format(severity_2/total))
print("Severity 3: ", severity_3, "{0:.2%} of total instances".format(severity_3/total))
print("Severity 4: ", severity_4, "{0:.2%} of total instances".format(severity_4/total))

1390073
Severity 1:  18920 1.36% of total instances
Severity 2:  1141065 82.09% of total instances
Severity 3:  136766 9.84% of total instances
Severity 4:  93322 6.71% of total instances


#### Stats for national data after deleting low counts

In [154]:
sub_df_new.Severity.value_counts()
severity_2, severity_3, severity_4, severity_1 = sub_df_new.Severity.value_counts()
total = len(sub_df_new.Severity)
print (total)
print("Severity 1: ", severity_1, "{0:.2%} of total instances".format(severity_1/total))
print("Severity 2: ", severity_2, "{0:.2%} of total instances".format(severity_2/total))
print("Severity 3: ", severity_3, "{0:.2%} of total instances".format(severity_3/total))
print("Severity 4: ", severity_4, "{0:.2%} of total instances".format(severity_4/total))

1329105
Severity 1:  18411 1.39% of total instances
Severity 2:  1095917 82.46% of total instances
Severity 3:  133813 10.07% of total instances
Severity 4:  80964 6.09% of total instances


#### Stats for TX data before cleaning

In [135]:
df.Severity[df.State == 'TX'].value_counts()
severity_2, severity_3, severity_4, severity_1 = df.Severity[df.State == 'TX'].value_counts()
total = len(df.Severity[df.State == 'TX'])
print (total)
print("Severity 1: ", severity_1, "{0:.2%} of total instances".format(severity_1/total))
print("Severity 2: ", severity_2, "{0:.2%} of total instances".format(severity_2/total))
print("Severity 3: ", severity_3, "{0:.2%} of total instances".format(severity_3/total))
print("Severity 4: ", severity_4, "{0:.2%} of total instances".format(severity_4/total))

149037
Severity 1:  941 0.63% of total instances
Severity 2:  124059 83.24% of total instances
Severity 3:  19570 13.13% of total instances
Severity 4:  4467 3.00% of total instances


#### Stats for TX data after cleaning

In [136]:
df_new.Severity[df_new.State == 'TX'].value_counts()
severity_2, severity_3, severity_4, severity_1 = df_new.Severity[df_new.State == 'TX'].value_counts()
total = len(df_new.Severity[df_new.State == 'TX'])
print (total)
print("Severity 1: ", severity_1, "{0:.2%} of total instances".format(severity_1/total))
print("Severity 2: ", severity_2, "{0:.2%} of total instances".format(severity_2/total))
print("Severity 3: ", severity_3, "{0:.2%} of total instances".format(severity_3/total))
print("Severity 4: ", severity_4, "{0:.2%} of total instances".format(severity_4/total))

61501
Severity 1:  656 1.07% of total instances
Severity 2:  40103 65.21% of total instances
Severity 3:  17055 27.73% of total instances
Severity 4:  3687 6.00% of total instances


#### Stats for TX data after deleting low counts

In [155]:
sub_df_new.Severity[sub_df_new.State == 'TX'].value_counts()
severity_2, severity_3, severity_4, severity_1 = sub_df_new.Severity[sub_df_new.State == 'TX'].value_counts()
total = len(sub_df_new.Severity[sub_df_new.State == 'TX'])
print (total)
print("Severity 1: ", severity_1, "{0:.2%} of total instances".format(severity_1/total))
print("Severity 2: ", severity_2, "{0:.2%} of total instances".format(severity_2/total))
print("Severity 3: ", severity_3, "{0:.2%} of total instances".format(severity_3/total))
print("Severity 4: ", severity_4, "{0:.2%} of total instances".format(severity_4/total))

59497
Severity 1:  640 1.08% of total instances
Severity 2:  38735 65.10% of total instances
Severity 3:  16812 28.26% of total instances
Severity 4:  3310 5.56% of total instances


#### Check cross street and street output

In [138]:
sub_df_new.value_counts(['Cross_street','Street']).nlargest(20) #San Bernandino Fwy/I-10 E pair has highest counts, confirmed from the raw data (below)

Cross_street          Street           
SAN BERNARDINO FWY E  I-10 E               3290
SAN DIEGO FWY S       I-405 S              2574
RIVERSIDE FWY E       CA-91 E              2034
POMONA FWY W          CA-60 W              1891
FL-94                 SW 88TH ST           1849
US-1 N                S DIXIE HWY          1810
FL-9                  NW 27TH AVE          1807
FL-825                SW 137TH AVE         1774
NIMITZ FWY S          I-880 S              1712
US-1 S                S DIXIE HWY          1651
FL-482                W SAND LAKE RD       1622
US-41 S               SW 8TH ST            1609
SANTA ANA FWY S       I-5 S                1564
FL-423                S JOHN YOUNG PKWY    1548
SAN DIEGO FWY S       I-5 S                1533
I-210 W               FOOTHILL FWY W       1532
RIVERSIDE FWY W       CA-91 W              1458
I-210 E               FOOTHILL FWY E       1428
I-95                  I-95 S               1391
FL-985                SW 107TH AVE         1389


#### Check from raw data

In [139]:
df.Street.value_counts().nlargest(5)

I-95 N    39853
I-5 N     39402
I-95 S    36425
I-5 S     30229
I-10 E    26164
Name: Street, dtype: int64

In [140]:
df_I95N = df[df.Street =='I-95 N']
df_I5N = df[df.Street =='I-5 N']
df_I95S = df[df.Street=='I-95 S']
df_I5S = df[df.Street =='I-5 S']
df_I10E = df[df.Street =='I-10 E']

In [141]:
df_I95N.Description.value_counts().nlargest(5)

INCIDENT ON I-95 NB NEAR I-95 DRIVE WITH CAUTION.                                                                   979
AT 151ST ST/EXIT 11 - ACCIDENT.                                                                                     140
INCIDENT ON I-95 NEAR I-95 DRIVE WITH CAUTION.                                                                      138
BETWEEN HUDSON TER/PALISADES PKY AND RT-9/US-9/HUDSON PKY/178TH ST - ACCIDENT.                                      129
SLOW TRAFFIC ON I-95 N FROM I-195/FL-112/AIRPORT EXPY (I-95 HOV) TO I-95 (MIAMI) (S) (I-95 HOV) DUE TO ACCIDENT.    127
Name: Description, dtype: int64

In [142]:
df_I5N.Description.value_counts().nlargest(5)

A CRASH HAS OCCURRED CAUSING NO TO MINIMUM DELAYS. USE CAUTION.    696
HAZARDOUS DEBRIS IS CAUSING NO TO MINIMUM DELAYS. USE CAUTION.     289
AT OR-214/EXIT 271 - ACCIDENT.                                     196
AT INTERSTATE BRIDGE - ACCIDENT.                                   191
AT OR-62/EXIT 30 - ACCIDENT.                                       188
Name: Description, dtype: int64

In [143]:
df_I95S.Description.value_counts().nlargest(5)

INCIDENT ON I-95 SB NEAR I-95 DRIVE WITH CAUTION.                                                                           977
SLOW TRAFFIC ON I-95 S FROM FL-816/NW 31ST ST/EXIT 31 (I-95) TO FL-842/BROWARD BLVD/EXIT 27 (I-95) DUE TO ACCIDENT.         258
SLOW TRAFFIC ON I-95 S FROM I-95 (S) (I-95 EXP) TO I-95 (MIAMI) (N) (I-95 HOV) DUE TO ACCIDENT.                             183
SLOW TRAFFIC ON I-95 S FROM I-95 EXP (I-95) TO I-395/FL-836/NE 3RD ST/NE 2ND AVE/EXIT 2D/EXIT 3A (I-95) DUE TO ACCIDENT.    161
SLOW TRAFFIC ON I-95 S FROM PECAN PARK RD/EXIT 366 (I-95) TO I-295/EXIT 362 (I-95) DUE TO ACCIDENT.                         156
Name: Description, dtype: int64

In [144]:
df_I5S.Description.value_counts().nlargest(5)

A CRASH HAS OCCURRED CAUSING NO TO MINIMUM DELAYS. USE CAUTION.    571
HAZARDOUS DEBRIS IS CAUSING NO TO MINIMUM DELAYS. USE CAUTION.     247
AT I-5 - ACCIDENT.                                                 222
AT OR-214/EXIT 271 - ACCIDENT.                                     207
A CRASH HAS OCCURRED USE CAUTION.                                  184
Name: Description, dtype: int64

In [145]:
#BERNARDINO FWY indeed has a high count in I-10 E accidents

n = 0
for i in df_I10E['Description']:
    i = i.strip().upper().split()
    if "BERNARDINO" in i:
        n = n+1
print(n)

4216
